# 3. Ecuaciones de Bellman

- Programación dinámica
- Value function iteration

### Programación dinámica

Planteemos un modelo de crecimiento neoclásico 

$$
\begin{array}{rl}
\max_{\{c_{t}, k_{t+1}\}_{t=0}^{\infty}} & \sum_{t=0}^{T}\beta^{t}u(c_{t}) \\ \\
\text{s.a} & c_{t}+k_{t+1} \leq f(k_{t}) \\ %\equiv F(k_{t}) + (1-\delta)k_{t}
		   & k_{0} > 0 \quad\text{dado}
\end{array}
$$

El problema puede resolverse de forma secuencial (ver [notebook 2.1](2.1-algorithms.ipynb)) o de forma recursiva (como veremos aquí).

El modelo de crecimiento neoclásico puede expresarse como un problema de programación dinámica de la siguiente manera:

$$
v(k) = \max_{k'\in\Gamma(k)}~~u[f(k) - k'] + \beta v(k')
$$

Lo anterior es una **Ecuación de Bellman**. Es una ecuación funcional y su solución es una función.

- $k$ es la variable de estado
- $k'$ es la variable de control
- $\Gamma(k)$ es el espacio de $k'$ factibles dado $k$

$v(k)$ representa la máxima utilidad que se puede obtener a partir de $k$ si se toman decisiones racionales.

Definimos a su vez la **función de política** $g(k)$ como el vector de variables de control que maximiza la trayectoria de utilidad, dada la condición inicial $k$.

$$
g(k) = \argmax_{k'\in\Gamma(k)}~~u[f(k) - k'] + \beta v(k')
$$

### Value function iteration

Una forma de resolver este problema es de forma recursiva. Aplicamos un algoritmo de iteración de función de valor. 

Supongamos que queremos resolver el problema de crecimiento planteado en la sección anterior, con una función de utilidad logarítmica y una función de producción Cobb-Douglas. 
$$
\begin{array}{rl}
u(c) &= log(c) \\
F(k) &= Ak^{\alpha} \\
f(k) &= F(k) + (1 - \delta)k
\end{array}
$$


Supongamos además que los parámetros de interés toman los siguientes valores: $\beta = 0.96, \delta = 1, A = 1, \alpha = 0.36$ y $k_0 = 0.3 * k^*$, donde $k^*$ es el valor de estado estacionario del capital. 

Inicializamos los parámetros y definimos las funciones

In [ ]:
using LinearAlgebra, Plots, LaTeXStrings

In [ ]:
α = 0.36
β = 0.96
δ = 0.05
A = 1.0

u(c) = log(c)
F(k) = A * k ^ α
f(k) = F(k) + (1 - δ) * k

#### Value function iteration

$$V^{i+1}(k) = \max_{k'\in\Gamma(k)} u(k) + \beta V^{i}(k')$$

1. Definir parámetros relevantes
	- `num_k`: número de puntos en la grilla de $k$
	- `k_min`: valor mínimo de $k$ factibles
	- `k_max`: valor máximo de $k$ factibles
	- `tol`: tolerancia de error (criterio de convergencia)
2. Definir grilla de $k$ (`num_k` puntos equidistantes entre `k_min` y `k_max`)
3. Definir punto de partida para la iteración
	- Una condición trivial de inicio es utilidad cero para todos los valores de $k$: $V^0 = \{0\}_{i=1}^{n_k}$
4. Actualizar iterativamente la función de valor $V^1$
	- Tomar un punto de la grilla $k^i$
	- Computar la utilidad de pasar del punto $k^i$ a todos los demás puntos de la grilla $k^j$.
	- Elegir el valor $k^j$ que genera la utilidad máxima partiendo de $k^i$ y guardarlo como $V^1(k^i)$. 
	- Repetir paso 4 para cada punto $k^i\in\Gamma$
5. Comparar $V^0$ y $V^1$. Computar distancia $d = ||V^1 - V^0||$
6. Si $d$ < `tol` entonces hay convergencia. Si $d$ > `tol` entonces repetimos el proceso con una nueva función $V^2$, usando $V^1$ como $V^0$ en la iteración anterior.


In [ ]:
plotV = plot(k_grid, V,
			 title = "Value function", 
			 xaxis = L"k",
			 yaxis = L"V(k)",
			 legend = false)
plotg = plot(k_grid, g,
			 color = "darkorange",
			 title = "Policy function",
			 xaxis = L"k",
			 yaxis = L"g(k)",
			 legend = false)
plot(plotV, plotg)